In [1]:
import requests
import json
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import re
import datetime
from datetime import datetime
import dateparser
import psycopg2
import smtplib
import pandas as pd
from requests.exceptions import RequestException


In [2]:

# Чтение текущей даты
current_time_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f"Привет! Текущая дата - {current_time_str}")


Привет! Текущая дата - 2024-09-10 11:05:30


In [3]:

# Счетчики
number = 0
page_counter = 1
page_generation = ''
stop_flag = False
df = pd.DataFrame(columns=['ID', 'Name', 'Date', 'Price RUB', 'Type', 'Seller', 'Description', 'Prim. Location', 'URL'])


In [4]:
cookies = {
    '_ym_uid': '1545027567707528589',
    '_ym_d': '1719303718',
    '_gcl_au': '1.1.1793393708.1719303719',
    '_ga': 'GA1.1.255704567.1719303719',
    'tb_favourites_hash': '171930371832733292',
    'cc_cookie': '{"categories":["necessary","analytics","targeting"],"level":["necessary","analytics","targeting"],"revision":0,"data":null,"rfc_cookie":false,"consent_date":"2024-06-25T08:22:22.588Z","consent_uuid":"0b05ba09-2875-4dc8-b667-50ab3b395a27","last_consent_update":"2024-06-25T08:22:22.588Z"}',
    'sbjs_migrations': '1418474375998%3D1',
    'sbjs_first_add': 'fd%3D2024-08-09%2010%3A24%3A14%7C%7C%7Cep%3Dhttps%3A%2F%2Fexkavator.ru%2Fmain%2Fnews%3Futm_source%3Dmain_page%26utm_medium%3Dmenu%26utm_campaign%3Dnews%7C%7C%7Crf%3Dhttps%3A%2F%2Fexkavator.ru%2Fmain%2Fnews%3Futm_source%3Dmain_page%26utm_medium%3Dmenu%26utm_campaign%3Dnews%26__cf_chl_tk%3DOlmPOphRNJNTsQM5UmeUiodI7gf3gTQsIIY7yGge1yw-1723188245-0.0.1.1-4884',
    'sbjs_first': 'typ%3Dutm%7C%7C%7Csrc%3Dmain_page%7C%7C%7Cmdm%3Dmenu%7C%7C%7Ccmp%3Dnews%7C%7C%7Ccnt%3D%28none%29%7C%7C%7Ctrm%3D%28none%29',
    'sbjs_current': 'typ%3Dorganic%7C%7C%7Csrc%3Dgoogle%7C%7C%7Cmdm%3Dorganic%7C%7C%7Ccmp%3D%28none%29%7C%7C%7Ccnt%3D%28none%29%7C%7C%7Ctrm%3D%28none%29',
    'sbjs_current_add': 'fd%3D2024-08-15%2011%3A43%3A41%7C%7C%7Cep%3Dhttps%3A%2F%2Fexkavator.ru%2Fexcapedia%2Ftechnic%2Frm-terextl155%7C%7C%7Crf%3Dhttps%3A%2F%2Fwww.google.com%2F',
    'v1_sessions_callibri': '920237060',
    'clbvid': '66c6c80a7e24badc9e6e8f8c',
    'callibri_start_date': '1724303371064',
    'callibri_current_page': 'https%3A//exkavator.ru/trade/search/TradeFlag/1/NoPrice/1/pages/',
    'callibri_page_counter': '29',
    'cf_clearance': 'oPDvGHacaaxOBptR7bZZjCmyYhPrve1moKNzeAbhG24-1725945431-1.2.1.1-R_jWEYuYQW9l8bpiANk_CV7XOAYEx.IrvPvcHYpS9q0us5.vP7tVAEGBW5OFqSP99tkz5f0gSUh8OzyDHjHXo0wdS8C2P6eqxmREGqGHz4nOIsubMQBe7cCb5AWXO0avJhX7rsxSHgNXGZMS.ykFVMpnX67WMP_vy1Ay6WfGSRZcio4kLzt20mHy6IX38VgKKu.HqImtGzE9jliJf2B0mPVQeMA7ndygm2beXrCbwv6BFEbuigK7Hw0_Z.E9_41PV01DE2rbi55wu.C0DKuX_JyZcsGR_AhOJofGwvhMuZtfhEi7UswjMhMf1jh400JtLtZJ0dEN_XEY1COR_bjgwBjNKowRsF2t6auf4R2s6S3GHLthf9Ns33PZMdWTo2ynSS5gERlPSfbHielzLhbN8wkLN6cuesaeAZmvvozMM7AMkbDm8s1CL2lKwxV5kzah',
    '_ym_isad': '1',
    'sbjs_udata': 'vst%3D51%7C%7C%7Cuip%3D%28none%29%7C%7C%7Cuag%3DMozilla%2F5.0%20%28Windows%20NT%2010.0%3B%20Win64%3B%20x64%29%20AppleWebKit%2F537.36%20%28KHTML%2C%20like%20Gecko%29%20Chrome%2F128.0.0.0%20Safari%2F537.36',
    'v1_referrer_callibri': 'https%3A//exkavator.ru/trade/search/TradeFlag/1/NoPrice/1/pages/%3F__cf_chl_tk%3DbFXOjJU_QMJEBJJrPXhb9P7HwXgOucbvjPyHq3iCwvs-1725945431-0.0.1.1-9556',
    'exkavator_session': 'a%3A5%3A%7Bs%3A10%3A%22session_id%22%3Bs%3A32%3A%22b7cb5fca0cd619acff16dc8e163481ae%22%3Bs%3A10%3A%22ip_address%22%3Bs%3A12%3A%2293.85.160.97%22%3Bs%3A10%3A%22user_agent%22%3Bs%3A111%3A%22Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F128.0.0.0+Safari%2F537.36%22%3Bs%3A13%3A%22last_activity%22%3Bi%3A1725952796%3Bs%3A9%3A%22user_data%22%3Bs%3A0%3A%22%22%3B%7Dcea228a96d0fac6efb511a450b95c0fa',
    '_ga_16VD85X6Z6': 'GS1.1.1725952793.35.0.1725952798.55.0.0',
}

headers = {
    'accept': 'text/css,*/*;q=0.1',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,be-BY;q=0.6,be;q=0.5',
    'cache-control': 'no-cache',
    # 'cookie': '_ym_uid=1545027567707528589; _ym_d=1719303718; _gcl_au=1.1.1793393708.1719303719; _ga=GA1.1.255704567.1719303719; tb_favourites_hash=171930371832733292; cc_cookie={"categories":["necessary","analytics","targeting"],"level":["necessary","analytics","targeting"],"revision":0,"data":null,"rfc_cookie":false,"consent_date":"2024-06-25T08:22:22.588Z","consent_uuid":"0b05ba09-2875-4dc8-b667-50ab3b395a27","last_consent_update":"2024-06-25T08:22:22.588Z"}; sbjs_migrations=1418474375998%3D1; sbjs_first_add=fd%3D2024-08-09%2010%3A24%3A14%7C%7C%7Cep%3Dhttps%3A%2F%2Fexkavator.ru%2Fmain%2Fnews%3Futm_source%3Dmain_page%26utm_medium%3Dmenu%26utm_campaign%3Dnews%7C%7C%7Crf%3Dhttps%3A%2F%2Fexkavator.ru%2Fmain%2Fnews%3Futm_source%3Dmain_page%26utm_medium%3Dmenu%26utm_campaign%3Dnews%26__cf_chl_tk%3DOlmPOphRNJNTsQM5UmeUiodI7gf3gTQsIIY7yGge1yw-1723188245-0.0.1.1-4884; sbjs_first=typ%3Dutm%7C%7C%7Csrc%3Dmain_page%7C%7C%7Cmdm%3Dmenu%7C%7C%7Ccmp%3Dnews%7C%7C%7Ccnt%3D%28none%29%7C%7C%7Ctrm%3D%28none%29; sbjs_current=typ%3Dorganic%7C%7C%7Csrc%3Dgoogle%7C%7C%7Cmdm%3Dorganic%7C%7C%7Ccmp%3D%28none%29%7C%7C%7Ccnt%3D%28none%29%7C%7C%7Ctrm%3D%28none%29; sbjs_current_add=fd%3D2024-08-15%2011%3A43%3A41%7C%7C%7Cep%3Dhttps%3A%2F%2Fexkavator.ru%2Fexcapedia%2Ftechnic%2Frm-terextl155%7C%7C%7Crf%3Dhttps%3A%2F%2Fwww.google.com%2F; v1_sessions_callibri=920237060; clbvid=66c6c80a7e24badc9e6e8f8c; callibri_start_date=1724303371064; callibri_current_page=https%3A//exkavator.ru/trade/search/TradeFlag/1/NoPrice/1/pages/; callibri_page_counter=29; cf_clearance=oPDvGHacaaxOBptR7bZZjCmyYhPrve1moKNzeAbhG24-1725945431-1.2.1.1-R_jWEYuYQW9l8bpiANk_CV7XOAYEx.IrvPvcHYpS9q0us5.vP7tVAEGBW5OFqSP99tkz5f0gSUh8OzyDHjHXo0wdS8C2P6eqxmREGqGHz4nOIsubMQBe7cCb5AWXO0avJhX7rsxSHgNXGZMS.ykFVMpnX67WMP_vy1Ay6WfGSRZcio4kLzt20mHy6IX38VgKKu.HqImtGzE9jliJf2B0mPVQeMA7ndygm2beXrCbwv6BFEbuigK7Hw0_Z.E9_41PV01DE2rbi55wu.C0DKuX_JyZcsGR_AhOJofGwvhMuZtfhEi7UswjMhMf1jh400JtLtZJ0dEN_XEY1COR_bjgwBjNKowRsF2t6auf4R2s6S3GHLthf9Ns33PZMdWTo2ynSS5gERlPSfbHielzLhbN8wkLN6cuesaeAZmvvozMM7AMkbDm8s1CL2lKwxV5kzah; _ym_isad=1; sbjs_udata=vst%3D51%7C%7C%7Cuip%3D%28none%29%7C%7C%7Cuag%3DMozilla%2F5.0%20%28Windows%20NT%2010.0%3B%20Win64%3B%20x64%29%20AppleWebKit%2F537.36%20%28KHTML%2C%20like%20Gecko%29%20Chrome%2F128.0.0.0%20Safari%2F537.36; v1_referrer_callibri=https%3A//exkavator.ru/trade/search/TradeFlag/1/NoPrice/1/pages/%3F__cf_chl_tk%3DbFXOjJU_QMJEBJJrPXhb9P7HwXgOucbvjPyHq3iCwvs-1725945431-0.0.1.1-9556; exkavator_session=a%3A5%3A%7Bs%3A10%3A%22session_id%22%3Bs%3A32%3A%22b7cb5fca0cd619acff16dc8e163481ae%22%3Bs%3A10%3A%22ip_address%22%3Bs%3A12%3A%2293.85.160.97%22%3Bs%3A10%3A%22user_agent%22%3Bs%3A111%3A%22Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F128.0.0.0+Safari%2F537.36%22%3Bs%3A13%3A%22last_activity%22%3Bi%3A1725952796%3Bs%3A9%3A%22user_data%22%3Bs%3A0%3A%22%22%3B%7Dcea228a96d0fac6efb511a450b95c0fa; _ga_16VD85X6Z6=GS1.1.1725952793.35.0.1725952798.55.0.0',
    'pragma': 'no-cache',
    'priority': 'u=0',
    'referer': 'https://exkavator.ru/',
    'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'style',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
}

In [16]:

while stop_flag == False:
    try:






            page_counter +=1
        
        page_generation = str(40*(page_counter-1))

    except RequestException as e:
        print(f"Произошла ошибка при запросе страницы {page_counter}: {e}")
        break


<Response [200]> https://exkavator.ru/trade/search/TradeFlag/1/NoPrice/1/pages/
Страница 2
 1
        id - 889776
        имя - Lonking CDM853GN
        цена - 7372119
        тип - Фронтальный погрузчик
        продавец - ГК Автодоркомплект
        описание - Только продажа техники! АРЕНДЫ НЕТ! Гарантия 12 месяцев или 2000 м/ч.
ЦЕНА С НДС!
Характеристики:
- Эксплуатационная масса, кг: 16200 ±300
- Номин
        дата - 2024-08-28 11:38:18
        год выпуска - 2023
        место - Москва 
        url - https://exkavator.ru/trade/lot/889776/2024-lonking_cdm853gn/
        ------------
        
 2
        id - 888004
        имя - VAN STE990.9
        цена - 88200000
        тип - Карьерный экскаватор
        продавец - ВАН СпецТех
        описание - Предлагаем к поставке карьерный экскаватор массой 98т VAN STE990.9 Цена с НДС. Под заказ. Не требует вложений. Готова к эксплуатации. Гарантия 12 меся
        дата - 2024-08-28 11:38:18
        год выпуска - 2023
        место - Москва 
     

TypeError: expected string or bytes-like object, got 'int'

In [5]:
url_page = f'https://exkavator.ru/trade/search/TradeFlag/1/NoPrice/1/pages/{page_generation}'
response_page = requests.get(url_page, headers=headers, cookies=cookies)
page = response_page.text
soup = BeautifulSoup(page, 'lxml')
print(response_page, url_page)

print (f'Страница {page_counter}')
names = soup.find_all("span", class_="title-tech")
prices = soup.select("a > span.info > span.price")
types = soup.find_all("span", class_="nw", style="color:#333;") 
sellers = soup.select("a > span.info > span.company-garant-years > span.company-name")
descriptions = soup.find_all("span", class_="desc")
dates = soup.find_all("span", class_="update-line", style="bottom: 8px;")
#infos = soup.find_all("div", class_="tech-info lot-click")
urls = soup.find_all("a", class_="link", target="_blank")
locs = soup.select ('a > span.info > span.city-year-mile > span > span > span')


<Response [200]> https://exkavator.ru/trade/search/TradeFlag/1/NoPrice/1/pages/
Страница 1


In [14]:

for name, price, type, seller, description, date, loc, url in zip(names, prices, types, sellers, descriptions, dates, locs, urls):
    # локация
    loc = (re.sub(r' и еще.*', '', loc.text).strip().replace(' ', ''))

    # дата
    date = date.text
    if date == 'Обновлено сегодня ':
        date = current_time_str
    else:
        date = dateparser.parse(date)
        #date_str = date.strftime('%Y-%m-%d %H:%M:%S')
        
    # подгонка и подрезка
    if price.text == 'Цена по запросу':
        price = 0
    else:
        price = re.sub(r'\sруб.', '', price.text).strip().replace(' ', '')
    #category = category.strip()
    
    name = name.text.strip()
    url = url.get("href")
    id = re.search(r'lot/(\d+)/', url)
    id = int(id.group(1))
    url = 'https://exkavator.ru' + url
    price = int(price)
    

    df.loc[len(df)] = (id, name, date, price, type, seller.text, description.text, loc, url)
    print(f""" {number}
id - {id}
имя - {name}
цена - {price}
тип - {type.text}
продавец - {seller.text}
описание - {description.text}
дата - {date}
место - {loc}
url - {url}
------------
""")
    number +=1

 74
id - 890286
имя - Мтз 82 с траншеекопателем ДЭМ - 150 (УДКМ-3)
цена - 3900000
тип - Трактор
продавец - ТПК МТЗ-Татарстан
описание - Пpедназнaчeн для пpокладки газо- и водопpовoдов, cетей кaнализации, связи и электpoпepeдачи в твердых и меpзлыx гpунтax, плaниpoвoчных и зeмляных pабо
дата - 2024-09-10 11:05:30
место - Елабуга
url - https://exkavator.ru/trade/lot/890286/2024-mtz_82_s_transheekopatelem_dem_-_150_udkm-3/
------------

 75
id - 890237
имя - Lonking CDM936N
цена - 3250000
тип - Фронтальный погрузчик
продавец - ДорАвтоСнаб, ТД
описание - Teхникa В нaличии(или в ближайшей поставке)
Цeна нa данную позицию указана соглacнo кoмплeктaции. Пoдобрать другую комплeктацию пoзиции мoжете oбрати
дата - 2024-09-10 11:05:30
место - Новосибирск
url - https://exkavator.ru/trade/lot/890237/2024-lonking_cdm936n/
------------

 76
id - 890238
имя - Lonking LG833N
цена - 4750000
тип - Фронтальный погрузчик
продавец - ДорАвтоСнаб, ТД
описание - Texника B наличии(или в ближайшей поставке)
Цeна

In [30]:
    
df_date = datetime.now().strftime('%Y-%m-%d')
df.to_excel(f'exc_ru_{df_date}.xlsx', index=False)